In [1]:

import polars as pl
from typing import Tuple, Optional

In [2]:

  #  """데이터를 로드하고 기본 정보를 출력하는 함수"""
df = pl.scan_parquet('data/maude_sample.parquet')
total_rows = df.select(pl.len()).collect().item()
total_cols = len(df.collect_schema().names())
print(f"전체 행: {total_rows:,}개, 전체 컬럼: {total_cols}개")
df

전체 행: 14,132,321개, 전체 컬럼: 2247개


### 기본 함수 정의

In [3]:
# ============================================================================
# 변환 함수 정의
# ============================================================================

def convert_previous_use_category(df):
    """
    previous_use_code를 previous_use_category로 변환 (문자형)
    - 'I' -> 'Initial'
    - 'R' -> 'Reuse'
    - 기타 (A, U, *, None) -> 'Unknown'
    """
    return df.with_columns(
        pl.when(pl.col("previous_use_code") == pl.lit("I"))
            .then(pl.lit("Initial"))
        .when(pl.col("previous_use_code") == pl.lit("R"))
            .then(pl.lit("Reuse"))
        .otherwise(pl.lit("Unknown"))
        .alias("previous_use_category")
    )


def convert_single_use_category(df):
    """
    single_use_flag를 문자열형으로 변환 (원본 컬럼 덮어쓰기)
    - 'N' 또는 '0' -> '0' (Non-single-use, 다회용)
    - 'Y' 또는 '1' -> '1' (Single-use, 일회용)
    - '2' -> 'Unknown'
    - 기타 (None, I 등) -> 'Unknown'
    """
    return df.with_columns(
        pl.when(pl.col("single_use_flag").cast(pl.Utf8, strict=False).str.strip_chars() == pl.lit('N'))
            .then(pl.lit('0'))
        .when(pl.col("single_use_flag").cast(pl.Utf8, strict=False).str.strip_chars() == pl.lit('Y'))
            .then(pl.lit('1'))
        .when(pl.col("single_use_flag").cast(pl.Utf8, strict=False).str.strip_chars() == pl.lit('0'))
            .then(pl.lit('0'))
        .when(pl.col("single_use_flag").cast(pl.Utf8, strict=False).str.strip_chars() == pl.lit('1'))
            .then(pl.lit('1'))
        .when(pl.col("single_use_flag").cast(pl.Utf8, strict=False).str.strip_chars() == pl.lit('2'))
            .then(pl.lit('Unknown'))
        .otherwise(pl.lit('Unknown'))
        .alias("single_use_flag")
    )


def convert_manufacturer_link_flag(df):
    """
    manufacturer_link_flag를 숫자형으로 변환 (원본 컬럼 덮어쓰기)
    - 'Y' -> 1
    - 기타 -> 0
    """
    return df.with_columns(
        pl.when(pl.col("manufacturer_link_flag") == pl.lit("Y"))
            .then(pl.lit(1))
            .otherwise(pl.lit(0))
            .alias("manufacturer_link_flag")
    )


def convert_single_use_flag(df):
    """
    single_use_flag를 문자열형으로 변환 (원본 컬럼 덮어쓰기)
    - 'N' 또는 '0' -> '0' (Non-single-use, 다회용)
    - 'Y' 또는 '1' -> '1' (Single-use, 일회용)
    - '2' -> 'Unknown'
    - 기타 (None, I 등) -> 'Unknown'
    """
    return df.with_columns(
        pl.when(pl.col("single_use_flag").cast(pl.Utf8, strict=False).str.strip_chars() == pl.lit('N'))
            .then(pl.lit('0'))
        .when(pl.col("single_use_flag").cast(pl.Utf8, strict=False).str.strip_chars() == pl.lit('Y'))
            .then(pl.lit('1'))
        .when(pl.col("single_use_flag").cast(pl.Utf8, strict=False).str.strip_chars() == pl.lit('0'))
            .then(pl.lit('0'))
        .when(pl.col("single_use_flag").cast(pl.Utf8, strict=False).str.strip_chars() == pl.lit('1'))
            .then(pl.lit('1'))
        .when(pl.col("single_use_flag").cast(pl.Utf8, strict=False).str.strip_chars() == pl.lit('2'))
            .then(pl.lit('Unknown'))
        .otherwise(pl.lit('Unknown'))
        .alias("single_use_flag")
    )


def check_single_use_consistency(df, verbose=True):
    """
    single_use_flag와 previous_use_category 일관성 검사 함수
    
    Parameters:
    -----------
    df : pl.LazyFrame or pl.DataFrame
        입력 데이터프레임
    verbose : bool
        결과 출력 여부 (기본값: True)
    
    Returns:
    --------
    dict
        분석 결과 딕셔너리
    """
    if verbose:
        print("="*70)
        print("single_use_flag vs previous_use_category 교차 분석")
        print("="*70)
    
    # 전체 개수 계산
    total_count = df.select(pl.len()).collect().item()
    
    if verbose:
        print(f"\n전체 데이터: {total_count:,}개")
    
    # 전체 교차표
    if verbose:
        print("\n[전체 교차표: single_use_flag vs previous_use_category]")
        print("="*70)
    
    cross_table = df.group_by(["single_use_flag", "previous_use_category"]).len().sort(["single_use_flag", "len"], descending=[False, True]).collect()
    
    if verbose:
        print("\n[개수]")
        print(cross_table)
        print("\n[비율 (%)]")
        for row in cross_table.iter_rows(named=True):
            single_use = row['single_use_flag']
            prev_use = row['previous_use_category']
            cnt = row['len']
            pct = (cnt / total_count * 100) if total_count > 0 else 0
            print(f"  single_use={single_use} + previous_use={prev_use}: {cnt:,}개 ({pct:.2f}%)")
    
    # 논리적으로 이상한 케이스: Single-use(1)인데 Reuse인 경우
    if verbose:
        print("\n" + "="*70)
        print("[논리적으로 이상한 케이스]")
        print("single_use_flag = '1' (Single-use)인데 previous_use_category = 'Reuse'인 경우:")
    
    weird_case = df.filter(
        (pl.col("single_use_flag") == '1') &
        (pl.col("previous_use_category") == "Reuse")
    )
    
    weird_count = weird_case.select(pl.len()).collect().item()
    weird_pct = (weird_count / total_count * 100) if total_count > 0 else 0
    
    if verbose:
        print(f"총 {weird_count:,}개 ({weird_pct:.2f}%)")
    
    return {
        'total_count': total_count,
        'weird_count': weird_count,
        'cross_table': cross_table
    }

In [4]:
def check_adverse_event_consistency(df, verbose=True):
    """
    event_type과 adverse_event_flag 일관성 검사 함수
    
    Parameters:
    -----------
    df : pl.LazyFrame or pl.DataFrame
        입력 데이터프레임
    verbose : bool
        결과 출력 여부 (기본값: True)
    
    Returns:
    --------
    dict
        분석 결과 딕셔너리
    """
    if verbose:
        print("="*70)
        print("event_type vs adverse_event_flag 교차 분석")
        print("="*70)
    
    # 전체 개수 계산
    total_count = df.select(pl.len()).collect().item()
    
    if verbose:
        print(f"\n전체 데이터: {total_count:,}개")
    
    # 1. Death/Injury인데 adverse_event_flag = N인 경우 (논리적으로 이상한 케이스)
    if verbose:
        print("\n[논리적으로 이상한 케이스]")
        print("event_type이 Death 또는 Injury인데 adverse_event_flag = N인 경우:")
    
    weird_case = df.filter(
        (pl.col("event_type").is_in(["Death", "Injury"])) &
        (pl.col("adverse_event_flag") == "N")
    )
    
    weird_count = weird_case.select(pl.len()).collect().item()
    weird_pct = (weird_count / total_count * 100) if total_count > 0 else 0
    
    if verbose:
        print(f"총 {weird_count:,}개 ({weird_pct:.2f}%)")
    
    if weird_count > 0 and verbose:
        print("\n상세 분석:")
        result = weird_case.group_by(["event_type", "adverse_event_flag"]).len().sort("len", descending=True).collect()
        for row in result.iter_rows(named=True):
            event = row['event_type']
            flag = row['adverse_event_flag']
            cnt = row['len']
            pct = (cnt / total_count * 100) if total_count > 0 else 0
            print(f"  {event} + {flag}: {cnt:,}개 ({pct:.2f}%)")
    
    # 2. Death/Injury인데 adverse_event_flag = Y인 경우 (정상 케이스)
    if verbose:
        print("\n" + "="*70)
        print("[정상 케이스]")
        print("event_type이 Death 또는 Injury인데 adverse_event_flag = Y인 경우:")
    
    normal_case = df.filter(
        (pl.col("event_type").is_in(["Death", "Injury"])) &
        (pl.col("adverse_event_flag") == "Y")
    )
    
    normal_count = normal_case.select(pl.len()).collect().item()
    normal_pct = (normal_count / total_count * 100) if total_count > 0 else 0
    
    if verbose:
        print(f"총 {normal_count:,}개 ({normal_pct:.2f}%)")
    
    if normal_count > 0 and verbose:
        print("\n상세 분석:")
        result = normal_case.group_by(["event_type", "adverse_event_flag"]).len().sort("len", descending=True).collect()
        for row in result.iter_rows(named=True):
            event = row['event_type']
            flag = row['adverse_event_flag']
            cnt = row['len']
            pct = (cnt / total_count * 100) if total_count > 0 else 0
            print(f"  {event} + {flag}: {cnt:,}개 ({pct:.2f}%)")
    
    # 3. 전체 교차표 (비율 포함)
    if verbose:
        print("\n" + "="*70)
        print("[전체 교차표: event_type vs adverse_event_flag]")
        print("="*70)
    
    cross_table = df.group_by(["event_type", "adverse_event_flag"]).len().sort(["event_type", "len"], descending=[False, True]).collect()
    
    if verbose:
        print("\n[개수]")
        print(cross_table)
        print("\n[비율 (%)]")
        for row in cross_table.iter_rows(named=True):
            event = row['event_type']
            flag = row['adverse_event_flag']
            cnt = row['len']
            pct = (cnt / total_count * 100) if total_count > 0 else 0
            print(f"  {event} + {flag}: {pct:.2f}%")
    
    return {
        'total_count': total_count,
        'weird_count': weird_count,
        'normal_count': normal_count,
        'cross_table': cross_table
    }


def analyze_manufacturer_link_vs_report_source(df, verbose=True):
    """
    manufacturer_link_flag별로 report_source_code가 
    'User facility report' 또는 'Distributor report'인 비율을 분석하는 함수
    
    Parameters:
    -----------
    df : pl.LazyFrame or pl.DataFrame
        입력 데이터프레임
    verbose : bool
        결과 출력 여부 (기본값: True)
    
    Returns:
    --------
    dict
        분석 결과 딕셔너리
    """
    if verbose:
        print("="*70)
        print("manufacturer_link_flag vs report_source_code 분석")
        print("="*70)
    
    # 전체 개수 계산
    total_count = df.select(pl.len()).collect().item()
    
    if verbose:
        print(f"\n전체 데이터: {total_count:,}개")
    
    # manufacturer_link_flag별로 그룹화
    if verbose:
        print("\n" + "="*70)
        print("[manufacturer_link_flag별 분석]")
        print("="*70)
    
    # manufacturer_link_flag별 전체 개수
    flag_counts = df.group_by("manufacturer_link_flag").len().sort("manufacturer_link_flag").collect()
    
    # manufacturer_link_flag별로 User facility report 또는 Distributor report인 경우
    target_sources = ['User facility report', 'Distributor report']
    target_df = df.filter(
        pl.col("report_source_code").is_in(target_sources)
    )
    
    # manufacturer_link_flag별로 교차 분석
    cross_table = df.group_by(["manufacturer_link_flag", "report_source_code"]).len().sort(
        ["manufacturer_link_flag", "len"], descending=[False, True]
    ).collect()
    
    if verbose:
        print("\n[전체 교차표: manufacturer_link_flag vs report_source_code]")
        print(cross_table)
    
    # manufacturer_link_flag별로 User facility report 또는 Distributor report 비율 계산
    if verbose:
        print("\n" + "="*70)
        print("[manufacturer_link_flag별 User facility report / Distributor report 비율]")
        print("="*70)
    
    results = []
    for row in flag_counts.iter_rows(named=True):
        flag_value = row['manufacturer_link_flag']
        flag_total = row['len']
        
        # 해당 flag_value에서 target_sources에 해당하는 개수
        target_count = target_df.filter(
            pl.col("manufacturer_link_flag") == flag_value
        ).select(pl.len()).collect().item()
        
        target_pct = (target_count / flag_total * 100) if flag_total > 0 else 0
        
        flag_label = "Y (Manufacturer linked)" if flag_value == 1 else "N (Not linked)"
        
        if verbose:
            print(f"\nmanufacturer_link_flag = {flag_value} ({flag_label}):")
            print(f"  전체: {flag_total:,}개")
            print(f"  User facility report 또는 Distributor report: {target_count:,}개 ({target_pct:.2f}%)")
        
        results.append({
            'manufacturer_link_flag': flag_value,
            'total_count': flag_total,
            'target_count': target_count,
            'target_pct': target_pct
        })
    
    # 전체 요약
    total_target = target_df.select(pl.len()).collect().item()
    total_target_pct = (total_target / total_count * 100) if total_count > 0 else 0
    
    if verbose:
        print("\n" + "="*70)
        print("[전체 요약]")
        print("="*70)
        print(f"전체 데이터: {total_count:,}개")
        print(f"User facility report 또는 Distributor report: {total_target:,}개 ({total_target_pct:.2f}%)")
    
    return {
        'total_count': total_count,
        'total_target_count': total_target,
        'total_target_pct': total_target_pct,
        'results': results,
        'cross_table': cross_table
    }

### 전처리 실행 코드

In [5]:
# ============================================================================
# 셀 2: 전처리 실행 (변환 결과 확인 포함)
# ============================================================================

import polars as pl

print("="*70)
print("전처리 시작")
print("="*70)

def show_conversion_results(df, column_name, before_df, before_column_name=None):
    """변환 전후 비교를 보여주는 함수
    
    Parameters:
    -----------
    df : pl.LazyFrame
        변환 후 데이터프레임
    column_name : str
        변환 후 컬럼명
    before_df : pl.LazyFrame
        변환 전 데이터프레임
    before_column_name : str, optional
        변환 전 컬럼명 (None이면 column_name과 동일)
    """
    if before_column_name is None:
        before_column_name = column_name
    
    print(f"\n{'='*50}")
    print(f"📊 {column_name} 변환 결과")
    print(f"{'='*50}")
    
    # 변환 전 (컬럼이 존재하는 경우에만)
    try:
        before_counts = before_df.select(
            pl.col(before_column_name).value_counts().alias('counts')
        ).collect().unnest('counts').sort('count', descending=True)
        print("\n변환 전:")
        print(before_counts)
    except Exception as e:
        print(f"\n변환 전: (컬럼 '{before_column_name}' 없음 - 새로 생성되는 컬럼)")
    
    # 변환 후
    after_counts = df.select(
        pl.col(column_name).value_counts().alias('counts')
    ).collect().unnest('counts').sort('count', descending=True)
    
    print("\n변환 후:")
    print(after_counts)

# 1. manufacturer_link_flag 변환
print("\n1 manufacturer_link_flag 변환 중...")
before_df = df
df = convert_manufacturer_link_flag(df)
show_conversion_results(df, 'manufacturer_link_flag', before_df)

# 2. previous_use_code 변환
print("\n2 previous_use_code 변환 중...")
before_df = df
df = convert_previous_use_category(df)
show_conversion_results(df, 'previous_use_category', before_df, before_column_name='previous_use_code')

# 3. single_use_flag 변환
print("\n3 single_use_flag 변환 중...")
before_df = df
df = convert_single_use_category(df)
show_conversion_results(df, 'single_use_flag', before_df)

print("\n" + "="*70)
print("전처리 완료")
print("="*70)

전처리 시작

1 manufacturer_link_flag 변환 중...

📊 manufacturer_link_flag 변환 결과

변환 전:
shape: (2, 2)
┌────────────────────────┬──────────┐
│ manufacturer_link_flag ┆ count    │
│ ---                    ┆ ---      │
│ str                    ┆ u32      │
╞════════════════════════╪══════════╡
│ Y                      ┆ 13269751 │
│ N                      ┆ 862570   │
└────────────────────────┴──────────┘

변환 후:
shape: (2, 2)
┌────────────────────────┬──────────┐
│ manufacturer_link_flag ┆ count    │
│ ---                    ┆ ---      │
│ i32                    ┆ u32      │
╞════════════════════════╪══════════╡
│ 1                      ┆ 13269751 │
│ 0                      ┆ 862570   │
└────────────────────────┴──────────┘

2 previous_use_code 변환 중...

📊 previous_use_category 변환 결과

변환 전:
shape: (7, 2)
┌───────────────────┬─────────┐
│ previous_use_code ┆ count   │
│ ---               ┆ ---     │
│ str               ┆ u32     │
╞═══════════════════╪═════════╡
│ I                 ┆ 6639524 │
│ U 

### 전처리 이후 확인용 (일관성 등) 
- adverse_event_flag와 event_type과 일관성
- manufacturer_link_flag와 report_source_code의 비율
- single_use_flag와 previous_use_code와 인관성

In [6]:
# ============================================================================
# 셀 3: 일관성 검사
# ============================================================================

print("\n" + "="*70)
print("일관성 검사")
print("="*70)

print("\n[검사 1] single_use_flag vs previous_use_category")
check_single_use_consistency(df)

print("\n\n[검사 2] adverse_event_flag vs event_type")
check_adverse_event_consistency(df)


일관성 검사

[검사 1] single_use_flag vs previous_use_category
single_use_flag vs previous_use_category 교차 분석

전체 데이터: 14,132,321개

[전체 교차표: single_use_flag vs previous_use_category]

[개수]
shape: (9, 3)
┌─────────────────┬───────────────────────┬─────────┐
│ single_use_flag ┆ previous_use_category ┆ len     │
│ ---             ┆ ---                   ┆ ---     │
│ str             ┆ str                   ┆ u32     │
╞═════════════════╪═══════════════════════╪═════════╡
│ 0               ┆ Unknown               ┆ 3297165 │
│ 0               ┆ Reuse                 ┆ 1875236 │
│ 0               ┆ Initial               ┆ 910083  │
│ 1               ┆ Initial               ┆ 5658429 │
│ 1               ┆ Unknown               ┆ 548714  │
│ 1               ┆ Reuse                 ┆ 7738    │
│ Unknown         ┆ Unknown               ┆ 1747877 │
│ Unknown         ┆ Initial               ┆ 71012   │
│ Unknown         ┆ Reuse                 ┆ 16067   │
└─────────────────┴───────────────────────┴────

{'total_count': 14132321,
 'weird_count': 113328,
 'normal_count': 5286690,
 'cross_table': shape: (16, 3)
 ┌────────────────────┬────────────────────┬─────────┐
 │ event_type         ┆ adverse_event_flag ┆ len     │
 │ ---                ┆ ---                ┆ ---     │
 │ str                ┆ str                ┆ u32     │
 ╞════════════════════╪════════════════════╪═════════╡
 │ null               ┆ N                  ┆ 386     │
 │ Death              ┆ Y                  ┆ 67154   │
 │ Death              ┆ N                  ┆ 1425    │
 │ Death              ┆ null               ┆ 28      │
 │ Injury             ┆ Y                  ┆ 5219536 │
 │ …                  ┆ …                  ┆ …       │
 │ No answer provided ┆ null               ┆ 392     │
 │ No answer provided ┆ Y                  ┆ 1       │
 │ Other              ┆ N                  ┆ 937     │
 │ Other              ┆ Y                  ┆ 47      │
 │ Other              ┆ null               ┆ 45      │
 └───────────

In [7]:
# ============================================================================
# 셀 3: 일관성 검사
# ============================================================================

print("\n" + "="*70)
print("일관성 검사")
print("="*70)

print("\n[검사 1] single_use_flag vs previous_use_category")
check_single_use_consistency(df)

print("\n\n[검사 2] adverse_event_flag vs event_type")
check_adverse_event_consistency(df)


일관성 검사

[검사 1] single_use_flag vs previous_use_category
single_use_flag vs previous_use_category 교차 분석

전체 데이터: 14,132,321개

[전체 교차표: single_use_flag vs previous_use_category]

[개수]
shape: (9, 3)
┌─────────────────┬───────────────────────┬─────────┐
│ single_use_flag ┆ previous_use_category ┆ len     │
│ ---             ┆ ---                   ┆ ---     │
│ str             ┆ str                   ┆ u32     │
╞═════════════════╪═══════════════════════╪═════════╡
│ 0               ┆ Unknown               ┆ 3297165 │
│ 0               ┆ Reuse                 ┆ 1875236 │
│ 0               ┆ Initial               ┆ 910083  │
│ 1               ┆ Initial               ┆ 5658429 │
│ 1               ┆ Unknown               ┆ 548714  │
│ 1               ┆ Reuse                 ┆ 7738    │
│ Unknown         ┆ Unknown               ┆ 1747877 │
│ Unknown         ┆ Initial               ┆ 71012   │
│ Unknown         ┆ Reuse                 ┆ 16067   │
└─────────────────┴───────────────────────┴────

{'total_count': 14132321,
 'weird_count': 113328,
 'normal_count': 5286690,
 'cross_table': shape: (16, 3)
 ┌────────────────────┬────────────────────┬─────────┐
 │ event_type         ┆ adverse_event_flag ┆ len     │
 │ ---                ┆ ---                ┆ ---     │
 │ str                ┆ str                ┆ u32     │
 ╞════════════════════╪════════════════════╪═════════╡
 │ null               ┆ N                  ┆ 386     │
 │ Death              ┆ Y                  ┆ 67154   │
 │ Death              ┆ N                  ┆ 1425    │
 │ Death              ┆ null               ┆ 28      │
 │ Injury             ┆ Y                  ┆ 5219536 │
 │ …                  ┆ …                  ┆ …       │
 │ No answer provided ┆ null               ┆ 392     │
 │ No answer provided ┆ Y                  ┆ 1       │
 │ Other              ┆ N                  ┆ 937     │
 │ Other              ┆ Y                  ┆ 47      │
 │ Other              ┆ null               ┆ 45      │
 └───────────

In [8]:
# ============================================================================
# 셀 5: manufacturer_link_flag vs report_source_code 분석
# ============================================================================

print("\n" + "="*70)
print("manufacturer_link_flag vs report_source_code 분석")
print("="*70)

analyze_manufacturer_link_vs_report_source(df)


manufacturer_link_flag vs report_source_code 분석
manufacturer_link_flag vs report_source_code 분석

전체 데이터: 14,132,321개

[manufacturer_link_flag별 분석]

[전체 교차표: manufacturer_link_flag vs report_source_code]
shape: (6, 3)
┌────────────────────────┬──────────────────────┬──────────┐
│ manufacturer_link_flag ┆ report_source_code   ┆ len      │
│ ---                    ┆ ---                  ┆ ---      │
│ i32                    ┆ str                  ┆ u32      │
╞════════════════════════╪══════════════════════╪══════════╡
│ 0                      ┆ Distributor report   ┆ 744841   │
│ 0                      ┆ Voluntary report     ┆ 87012    │
│ 0                      ┆ User Facility report ┆ 30717    │
│ 1                      ┆ Manufacturer report  ┆ 13259988 │
│ 1                      ┆ Distributor report   ┆ 9567     │
│ 1                      ┆ User Facility report ┆ 196      │
└────────────────────────┴──────────────────────┴──────────┘

[manufacturer_link_flag별 User facility report / D

{'total_count': 14132321,
 'total_target_count': 754408,
 'total_target_pct': 5.338174812191147,
 'results': [{'manufacturer_link_flag': 0,
   'total_count': 862570,
   'target_count': 744841,
   'target_pct': 86.35136858457865},
  {'manufacturer_link_flag': 1,
   'total_count': 13269751,
   'target_count': 9567,
   'target_pct': 0.07209630384172243}],
 'cross_table': shape: (6, 3)
 ┌────────────────────────┬──────────────────────┬──────────┐
 │ manufacturer_link_flag ┆ report_source_code   ┆ len      │
 │ ---                    ┆ ---                  ┆ ---      │
 │ i32                    ┆ str                  ┆ u32      │
 ╞════════════════════════╪══════════════════════╪══════════╡
 │ 0                      ┆ Distributor report   ┆ 744841   │
 │ 0                      ┆ Voluntary report     ┆ 87012    │
 │ 0                      ┆ User Facility report ┆ 30717    │
 │ 1                      ┆ Manufacturer report  ┆ 13259988 │
 │ 1                      ┆ Distributor report   ┆ 9567  

In [9]:
# ============================================================================
# 셀 4: 변환 결과 확인
# ============================================================================

# 변환된 컬럼들 확인
print("\n" + "="*70)
print("변환 결과 확인")
print("="*70)

result = df.select([
    'manufacturer_link_flag',
    'previous_use_code',
    'previous_use_category',
    'single_use_flag',
    'adverse_event_flag',
    'event_type'
]).collect().head(10)

print(result)


변환 결과 확인
shape: (10, 6)
┌────────────────┬────────────────┬────────────────┬────────────────┬────────────────┬─────────────┐
│ manufacturer_l ┆ previous_use_c ┆ previous_use_c ┆ single_use_fla ┆ adverse_event_ ┆ event_type  │
│ ink_flag       ┆ ode            ┆ ategory        ┆ g              ┆ flag           ┆ ---         │
│ ---            ┆ ---            ┆ ---            ┆ ---            ┆ ---            ┆ str         │
│ i32            ┆ str            ┆ str            ┆ str            ┆ str            ┆             │
╞════════════════╪════════════════╪════════════════╪════════════════╪════════════════╪═════════════╡
│ 1              ┆ U              ┆ Unknown        ┆ 1              ┆ N              ┆ Malfunction │
│ 1              ┆ R              ┆ Reuse          ┆ 0              ┆ N              ┆ Malfunction │
│ 1              ┆ R              ┆ Reuse          ┆ 0              ┆ N              ┆ Malfunction │
│ 1              ┆ R              ┆ Reuse          ┆ 0            